# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication:
```name: my_project_acs_connection_credentialless
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net
metadata:
  ApiType: Azure
  location: East US
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-ai-search-fvye
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [ ]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

In [ ]:
conn_id

In [ ]:
# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

In [ ]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

In [ ]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="What kind of keys does the the MS Surface Book keyboard include?" # "How does wikipedia explain Euler's Identity?"
)
print(f"Created message: {message}")

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

In [ ]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
messages

In [ ]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

In [ ]:
# Print citations from the messages
for annotation in messages.file_citation_annotations:
    print(annotation["text"], annotation["file_citation"]["file_id"])

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

In [ ]:
for rs in run_steps["data"]:
    print(rs, '\n')

# START teardown

In [43]:
project_client.agents.list_agents()

DecodeError: JSON is invalid: Expecting value: line 1 column 1 (char 0)
Content: Exception Type: ServiceInvocationException
Message: Service invocation failed!
Request: GET account-rp.vienna-eastus.svc/rp/workspaces/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch
Status Code: 404 NotFound
Error Code: UserError/NotFoundError
Reason Phrase: Connection mmai-hub04-fvye-connection-AISearch can't be found in this workspace
Response Body: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "Connection mmai-hub04-fvye-connection-AISearch can't be found in this workspace",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo"

In [37]:
for f in project_client.agents.list_vector_stores()['data']:
    project_client.agents.delete_vector_store(f.id)

DecodeError: JSON is invalid: Expecting value: line 1 column 1 (char 0)
Content: Exception Type: ServiceInvocationException
Message: Service invocation failed!
Request: GET account-rp.vienna-eastus.svc/rp/workspaces/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch
Status Code: 404 NotFound
Error Code: UserError/NotFoundError
Reason Phrase: Connection mmai-hub04-fvye-connection-AISearch can't be found in this workspace
Response Body: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "Connection mmai-hub04-fvye-connection-AISearch can't be found in this workspace",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo"

In [6]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

# HIC SUNT LEONES